# Driver Identification and Transport Mode Classification using ResNet50-GRU and BiLSTM MTL model
## (Hyperparam Tuning)
To better understand the code, check out `./prep_files/MultiTaskModel.ipynb`. That notebook explains the thought process behind this code on a sample dataset whilst this goes straight to the point. 

In [1]:
import numpy as np
import pandas as pd
import torch
from torchsummary import summary
from torch.utils.data import DataLoader, random_split, Subset

from src.engine import MTL_engine
from src.model_multitask import MultitaskModel
from src.plot import plot_history
from src.dataset import CombinedDataset
from src.hyperparam import RayTuning
from ray import tune, train

from IPython.display import display, HTML

In [3]:
# Load custom CSS file
css = HTML('<link rel="stylesheet" type="text/css" href="./custom.css">')  # Ensure the path is correct
display(css)

In [4]:
model = MultitaskModel(input_size=6, hidden_size=512, num_layers=2)
summary(model, verbose=0)

Layer (type:depth-idx)                   Param #
├─BiLSTMNetwork: 1-1                     --
|    └─LSTM: 2-1                         8,429,568
├─ResNet50_GRU: 1-2                      --
|    └─Sequential: 2-2                   --
|    |    └─Conv2d: 3-1                  (9,408)
|    |    └─BatchNorm2d: 3-2             (128)
|    |    └─ReLU: 3-3                    --
|    |    └─MaxPool2d: 3-4               --
|    |    └─Sequential: 3-5              (215,808)
|    |    └─Sequential: 3-6              (1,219,584)
|    |    └─Sequential: 3-7              7,098,368
|    |    └─Sequential: 3-8              14,964,736
|    └─BatchNorm2d: 2-3                  4,096
|    └─Dropout: 2-4                      --
|    └─GRU: 2-5                          5,511,168
├─Linear: 1-3                            786,944
├─Linear: 1-4                            262,656
├─ReLU: 1-5                              --
├─GRU: 1-6                               3,151,872
├─GRU: 1-7                               3

In [5]:
model

MultitaskModel(
  (lstm_network): BiLSTMNetwork(
    (lstm): LSTM(6, 512, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  )
  (resnet_gru_network): ResNet50_GRU(
    (resnet50): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [6]:
from sklearn.utils.class_weight import compute_class_weight

import torch.nn as nn
import torch.optim as optim

#### Computing class weights for both tasks

driver identification

In [7]:
y_train = pd.read_csv('./data/feature_maps_labels/train/metadata.csv')
y_train = y_train.iloc[:,1].values
print('Classes:', np.unique(y_train))

class_weights_dr = torch.tensor(compute_class_weight('balanced', classes=np.unique(y_train), y=y_train), dtype=torch.float32)
class_weights_dr = class_weights_dr / class_weights_dr.sum()
class_weights_dr = class_weights_dr.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

print('Class weights:', class_weights_dr)
del y_train

Classes: [0 1 2 3]
Class weights: tensor([0.0153, 0.2646, 0.1717, 0.5485], device='cuda:0')


transport mode classification

In [8]:
y_train = pd.read_csv('./data/lstm_features_labels/train/metadata.csv')
y_train = y_train.iloc[:,1].values
print('Classes:', np.unique(y_train))

class_weights_tr = torch.tensor(compute_class_weight('balanced', classes=np.unique(y_train), y=y_train), dtype=torch.float32)
class_weights_tr = class_weights_tr / class_weights_tr.sum()
class_weights_tr = class_weights_tr.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

print('Class weights:', class_weights_tr)
del y_train

Classes: [0. 1. 2. 3. 4. 5. 6. 7.]
Class weights: tensor([0.0521, 0.0592, 0.4732, 0.1143, 0.0655, 0.1070, 0.0639, 0.0648],
       device='cuda:0')


In [9]:
lstm_base_dir = './data/lstm_features_labels/'
fmap_base_dir = './data/feature_maps_labels/'

train_datasets = CombinedDataset(fmap_base_dir, lstm_base_dir, mode='train', 
                                 rescale=False, augment=True)   #noRot augmentation
val_test_datasets = CombinedDataset(fmap_base_dir, lstm_base_dir, mode='valid',
                                 rescale=False, augment=True)   #noRot augmentation

np.random.seed(42)
valid_datasets, test_datasets = random_split(val_test_datasets, [0.5, 0.5])  

In [10]:
import random
np.random.seed(2)

train_datasets = Subset(train_datasets, random.sample(range(46240), 4624))
valid_datasets = Subset(valid_datasets, random.sample(range(5782), 578))
test_datasets = Subset(test_datasets, random.sample(range(5781), 578))

In [11]:
len(train_datasets), len(valid_datasets), len(test_datasets)

(4624, 578, 578)

### Hyperparameter Tuning

In [12]:
# Enhanced Hyperparameter search space for Multitask Model
config = {
    "optimizer": tune.choice(["adam", "adamw"]),
    "lr": tune.loguniform(1e-4, 1e-2),
    "scheduler": tune.choice(["exp", "ReduceLROnPlateau"]),
    "gamma": tune.uniform(0.3, 0.95),  # for exponential scheduler, wider range
    "patience": tune.choice([5, 10, 15]),  # for ReduceLROnPlateau
    "epochs": tune.choice([30]),  # Starting with a wider range
    "batch_size": tune.choice([32, 64, 128]),
    "weight_decay": tune.loguniform(1e-4, 0.1),  # Slightly wider range
    "hidden_size": tune.choice([256, 300, 512, 1024]),  # Including 300 and ranges around it
    "num_layers": tune.choice([1, 2, 3]),  # Number of GRU layers
    "dropout": tune.uniform(0.3, 0.7),  # Dropout rate, ensuring valid values for num_layers
    "alpha": tune.choice([1]),
    "beta": tune.choice([1])
}

criterion_driver = nn.CrossEntropyLoss(weight=class_weights_dr)
criterion_transport = nn.CrossEntropyLoss(weight=class_weights_tr)
criterion = [criterion_driver, criterion_transport]

save_dir = 'MultiTaskModel'
model = MultitaskModel
modelType = 'MultiTaskModel'
engine = MTL_engine

In [13]:
mytuner = RayTuning(config, save_dir, criterion, model, modelType, engine)

In [14]:
mytuner.main(train_datasets, valid_datasets, test_datasets, num_samples=40)

(train_model pid=28696) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/MultiTaskModel/trial_89fac_00000/checkpoint_000000)
(train_model pid=28696) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/MultiTaskModel/trial_89fac_00000/checkpoint_000001)
(train_model pid=28696) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/MultiTaskModel/trial_89fac_00000/checkpoint_000002)
(train_model pid=28696) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Re

Best trial config: {'optimizer': 'adamw', 'lr': 0.0005392288692948486, 'scheduler': 'ReduceLROnPlateau', 'gamma': 0.7511991545263691, 'patience': 15, 'epochs': 30, 'batch_size': 128, 'weight_decay': 0.0422480267454383, 'hidden_size': 1024, 'num_layers': 1, 'dropout': 0.6409051706390204, 'alpha': 1, 'beta': 1}
Best trial final validation loss: 2.084122633934021
Best trial final validation accuracy: Driver77.3356%, Transport44.8097%


AttributeError: 'bool' object has no attribute 'sum'

In [ ]:
model

In [ ]:
np.random.seed(42)

batch_size = 64
train_dl = DataLoader(train_datasets, batch_size, shuffle=True, num_workers=4)
valid_dl = DataLoader(valid_datasets, batch_size, shuffle=True, num_workers=4)
test_dl = DataLoader(test_datasets, batch_size, shuffle=False, num_workers=2)

In [12]:
criterion_driver = nn.CrossEntropyLoss(weight=class_weights_dr)
criterion_transport = nn.CrossEntropyLoss(weight=class_weights_tr)
model = MultitaskModel(6, 512, 2)
optimizer = optim.Adam(model.parameters(), lr=0.0003872118032174588)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.13618183112843046)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

In [13]:
engine = MTL_engine(model, optimizer, scheduler, criterion_driver, criterion_transport, device)

In [ ]:
hist = engine.train_validation(train_dl, valid_dl, epochs=2, save_path=None)